<a href="https://colab.research.google.com/github/marianoogimenez/google-asl-project/blob/master/Paywall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from google.cloud import bigquery
from google.colab import auth

In [0]:
auth.authenticate_user()

In [0]:
client = bigquery.Client(project='qwiklabs-gcp-4a684069c4776675')
sql="""
SELECT pase_id,dias_navegacion_x,n_visitas_x,n_disp_x,v_pw_x,q_notas_x,q_homes_x,pv_deportes_x,
       pv_espectaculos_x,pv_estilo_x,pv_noticias_x,pv_servicios_x,pv_sudoku_x,pv_sociedad_x,
       v_deportes_x,v_espectaculos_x,v_estilo_x,v_noticias_x,v_servicios_x,v_sudoku_x,
       v_sociedad_x,pvd_n,a_edad,susc_target,antiguedad_pase,q_visitas_10,q_visitas,
       genero_n,CLUSTER_ID,Unnamed,dias_navegacion_y,n_visitas_y,pvd,n_disp_y,v_pw_y,q_notas_y,
       q_homes_y,pv_deportes_y,pv_espectaculos_y,pv_estilo_y,pv_noticias_y,pv_servicios_y,pv_sudoku_y,
       pv_sociedad_y,v_deportes_y,v_espectaculos_y,v_estilo_y,v_noticias_y,v_servicios_y,v_sudoku_y,v_sociedad_y
FROM `qwiklabs-gcp-4a684069c4776675.AGEA_ASL.Paywall` 
"""

df = client.query(sql).to_dataframe()
df.head()

,pase_id,dias_navegacion_x,n_visitas_x,n_disp_x,v_pw_x,q_notas_x,q_homes_x,pv_deportes_x,pv_espectaculos_x,pv_estilo_x,...,pv_servicios_y,pv_sudoku_y,pv_sociedad_y,v_deportes_y,v_espectaculos_y,v_estilo_y,v_noticias_y,v_servicios_y,v_sudoku_y,v_sociedad_y
0,3628102,2,1,1,0,0,0,0,0,0,...,2.064900e-35,3.303841e-34,0.0,0.000000,0.0,2.643072e-33,0.000000,-4.129801e-35,6.607681e-34,0.0
1,6489400,10,1,1,0,0,12,0,0,0,...,2.064900e-35,3.303841e-34,0.0,0.000000,0.0,2.643072e-33,-0.072703,-4.129801e-35,6.607681e-34,0.0
2,1455866,5,1,1,0,0,0,0,0,0,...,2.064900e-35,-6.251681e-02,0.0,0.000000,0.0,2.643072e-33,0.000000,-4.129801e-35,-3.754676e-01,0.0
3,4575019,10,1,1,0,0,30,7,0,0,...,2.064900e-35,3.303841e-34,0.0,-0.071660,0.0,-3.775897e-01,-0.218108,-4.129801e-35,6.607681e-34,0.0
4,3909939,6,1,1,0,0,0,0,0,0,...,2.064900e-35,4.376177e-01,0.0,-0.143319,0.0,2.643072e-33,0.000000,-4.129801e-35,0.000000e+00,0.0


In [0]:
df.columns

Index(['dias_navegacion_x', 'n_disp_x', 'v_pw_x', 'q_notas_x', 'q_homes_x',
       'pv_deportes_x', 'pv_espectaculos_x', 'pv_estilo_x', 'pv_noticias_x',
       'pv_servicios_x', 'pv_sudoku_x', 'pv_sociedad_x', 'v_deportes_x',
       'v_espectaculos_x', 'v_estilo_x', 'v_noticias_x', 'v_servicios_x',
       'v_sudoku_x', 'v_sociedad_x', 'a_edad', 'susc_target',
       'antiguedad_pase', 'q_visitas_10', 'q_visitas', 'genero_n',
       'CLUSTER_ID', 'dias_navegacion_y', 'pvd', 'n_disp_y', 'v_pw_y',
       'q_notas_y', 'q_homes_y', 'pv_deportes_y', 'pv_espectaculos_y',
       'pv_estilo_y', 'pv_noticias_y', 'pv_servicios_y', 'pv_sudoku_y',
       'pv_sociedad_y', 'v_deportes_y', 'v_espectaculos_y', 'v_estilo_y',
       'v_noticias_y', 'v_servicios_y', 'v_sudoku_y', 'v_sociedad_y'],
      dtype='object')

In [0]:
df = df.drop(["pase_id","pvd_n","n_visitas_x","n_visitas_y",'Unnamed"],axis = 1)
